# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import pickle
import pandas as pd
pd.set_option('display.max_columns', 200)

from card import Card
from board import Board
from player import Player
from rule import Rule
import optimal_strategy as opt

import multiprocessing
import time
from multiprocessing.pool import ThreadPool
import threading
from tqdm import notebook
notebook.tqdm.pandas()

from euchre_lib import *
from search_lib import *
import data_formatting as dtf

In [3]:
notebook.tqdm.get_lock().locks = []

# Create dataset

In [211]:
os.system('rm -r RLdataset/*')

0

In [212]:
mx_thresh = 72; N_CPUS = 8
thresholds = [mx_thresh]*N_CPUS
n_hands = int(1e4)
n_epochs = 100
# get 8 million = 8e6 = 8 CPUS x 1e4 hands x 1e2 epochs
# estimate it takes 35 minutes, given 500 hands/sec
# actually more like 90-100 minutes, eh
if not os.path.exists(os.path.join(os.getcwd(), 'RLdataset')):
    os.mkdir(os.path.join(os.getcwd(), 'RLdataset'))

In [213]:
parallel_search_wrapper(thresholds, MAX_CPUS=N_CPUS, n_epochs=n_epochs, n_hands=n_hands,\
                        opp_thresh=mx_thresh, title='Making dataset', folder='RLdataset')

Making dataset:   0%|          | 0/100 [00:00<?, ?it/s]

Making dataset:   0%|          | 0/100 [00:00<?, ?it/s]

Making dataset:   0%|          | 0/100 [00:00<?, ?it/s]

Making dataset:   0%|          | 0/100 [00:00<?, ?it/s]

Making dataset:   0%|          | 0/100 [00:00<?, ?it/s]

Making dataset:   0%|          | 0/100 [00:00<?, ?it/s]

Making dataset:   0%|          | 0/100 [00:00<?, ?it/s]

Making dataset:   0%|          | 0/100 [00:00<?, ?it/s]

Done!
This took 4722.23 seconds
This took 78.70 minutes


In [ ]:
#def parallel_search_wrapper(thresholds, MAX_CPUS=8, n_epochs=10, n_hands=100, opp_thresh=None,
#                            title=None, ROOT_DIR=os.getcwd()+'/', folder=None):

# Predict Expert Moves

In [4]:
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

2.4.0


In [6]:
infolder = os.path.join(os.getcwd(), 'RLdataset')
#infolder = os.path.join(os.getcwd(), 'thresholds', 'thresh70')
data = read_all_hands(infolder)
data.shape

  0%|          | 0/100 [00:00<?, ?it/s]

(1000000, 122)

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Columns: 122 entries, p0c1 to p3trueid
dtypes: int64(101), object(21)
memory usage: 930.8+ MB


In [8]:
data.head()

,p0c1,p0c1isD,p0c1isH,p0c1isS,p0c2,p0c2isD,p0c2isH,p0c2isS,p0c3,p0c3isD,p0c3isH,p0c3isS,p0c4,p0c4isD,p0c4isH,p0c4isS,p0c5,p0c5isD,p0c5isH,p0c5isS,p1c1,p1c1isD,p1c1isH,p1c1isS,p1c2,p1c2isD,p1c2isH,p1c2isS,p1c3,p1c3isD,p1c3isH,p1c3isS,p1c4,p1c4isD,p1c4isH,p1c4isS,p1c5,p1c5isD,p1c5isH,p1c5isS,p2c1,p2c1isD,p2c1isH,p2c1isS,p2c2,p2c2isD,p2c2isH,p2c2isS,p2c3,p2c3isD,p2c3isH,p2c3isS,p2c4,p2c4isD,p2c4isH,p2c4isS,p2c5,p2c5isD,p2c5isH,p2c5isS,p3c1,p3c1isD,p3c1isH,p3c1isS,p3c2,p3c2isD,p3c2isH,p3c2isS,p3c3,p3c3isD,p3c3isH,p3c3isS,p3c4,p3c4isD,p3c4isH,p3c4isS,p3c5,p3c5isD,p3c5isH,p3c5isS,TC_power,TC_isD,TC_isH,TC_isS,trump_isD,trump_isH,trump_isS,caller,round,alone,winner1,winner2,winner3,winner4,winner5,points02,points13,played1,played2,played3,played4,played5,played6,played7,played8,played9,played10,played11,played12,played13,played14,played15,played16,played17,played18,played19,played20,result,p0trueid,p1trueid,p2trueid,p3trueid
0,20,0,1,0,12,0,1,0,10,0,0,0,1,0,0,0,2,0,0,1,35,0,1,0,31,1,0,0,4,0,0,0,3,0,0,0,1,0,0,1,10,1,0,0,4,1,0,0,1,1,0,0,5,0,0,1,4,0,0,1,30,0,1,0,5,1,0,0,2,1,0,0,10,0,0,1,3,0,0,1,15,0,1,0,0,1,0,0,1,0,0,0,3,0,3,1,0,JH0,9D1,AH2,9H3,JD0,QD1,TD2,TH3,QC0,QS1,JS2,AC3,9C3,JC0,KS1,KD2,9S0,AD1,AS2,QH3,Single,3,0,1,2
1,30,0,0,0,15,0,0,0,4,0,1,0,2,0,1,0,10,0,0,1,20,0,0,0,5,1,0,0,2,1,0,0,5,0,0,1,4,0,0,1,31,0,0,1,10,1,0,0,3,1,0,0,5,0,1,0,2,0,0,1,35,0,0,0,12,0,0,0,1,1,0,0,3,0,1,0,1,0,1,0,25,0,0,0,0,0,0,1,1,0,0,3,3,3,3,0,1,JC0,TC1,QC2,KC3,JH0,QH1,TD2,KH3,AD3,9D0,TH1,KD2,JS3,9C0,AC1,QS2,JD3,9H0,AS1,KS2,Single,0,1,2,3
2,3,0,0,0,5,0,1,0,10,0,0,1,5,0,0,1,1,0,0,1,30,1,0,0,25,1,0,0,20,1,0,0,12,1,0,0,2,0,1,0,31,0,1,0,10,0,0,0,2,0,0,0,4,0,1,0,2,0,0,1,35,1,0,0,4,0,0,0,1,0,0,0,1,0,1,0,3,0,0,1,15,1,0,0,1,0,0,0,1,0,2,0,0,0,0,2,0,AD0,JH1,JD2,TD3,QC2,JC3,9D0,TC1,KD0,TS1,9C2,KH3,QD0,QH1,9H2,KS3,TH0,AC1,JS2,AS3,Sweep,1,2,3,0
3,35,0,0,0,30,0,0,0,12,0,0,0,10,1,0,0,1,0,1,0,5,1,0,0,4,1,0,0,10,0,1,0,4,0,1,0,10,0,0,1,15,0,0,0,2,1,0,0,3,0,1,0,5,0,0,1,4,0,0,1,31,0,0,1,3,1,0,0,5,0,1,0,2,0,1,0,2,0,0,1,25,0,0,0,0,0,0,1,1,0,2,1,1,1,1,0,1,KH0,9H1,AH2,JH3,AS2,QS3,TS0,9C1,JC1,QD2,TC3,JS0,AD1,KD2,KS3,JD0,AC1,QH2,KC3,TH0,Single,2,3,0,1
4,10,0,0,0,4,0,0,0,10,1,0,0,2,0,1,0,1,0,1,0,5,1,0,0,4,1,0,0,1,1,0,0,5,0,1,0,4,0,1,0,31,0,0,0,25,0,0,1,2,0,0,0,1,0,0,0,3,0,1,0,30,0,0,1,20,0,0,1,12,0,0,1,5,0,0,0,2,1,0,0,35,0,0,1,0,0,1,1,1,0,1,2,3,3,2,0,1,KD0,KS1,TD2,AD3,JH1,9S2,TH3,QH0,KC2,AC3,9D0,9C1,JS3,QD0,JC1,QS2,QC3,KH0,TC1,AS2,Single,3,0,1,2


In [9]:
p0hand = ['JH', 'JD', 'QC', 'JC', '9S']
p1hand = ['AD', 'QD', '9D', 'KS', 'QS']
p2hand = ['AH', 'KD', 'TD', 'AS', 'JS']
p3hand = ['QH', '9H', 'AC', '9C', 'TS'] + ['TH']

In [159]:
# this seems to be working just fine
group1_info = dtf.get_group1_info(data.iloc[1], 0, 0)
for arr in [group1_info[:4], group1_info[4:8], group1_info[8], group1_info[9], group1_info[10:14], group1_info[14:]]:
    print(arr)
z="""
    #######
    DATA GROUP 1: Calling trump
    #######
    (4)  1.) Who dealt (position relative to "me")
    (4)  2.) Who called trump (position relative to "me")
    (1)  3.) Which round was trump called in
    (1)  4.) Going alone?
    (4)  5.) Which suit is trump (not sure if this one needs to be here)
    (10) 6.) What is the turn card
    Total: 24 columns
    """

[0 0 1 0]
[1 0 0 0]
1
0
[1 0 0 0]
[0 0 0 0 1 0 1 0 0 0]


In [149]:
# this seems to be working just fine
tnum = 4
group2_info = dtf.get_group2_info(data.iloc[0], tnum, 3)
for arr in [group2_info[:4], group2_info[4:8], group2_info[8:19], group2_info[19:30], group2_info[30:35], group2_info[35:]]:
    print(arr)
print('Led card:', data.iloc[0]['played'+str(4*tnum+1)])

z="""
    #######
    DATA GROUP 2: Other misc. information
    #######
    (4)  1.) Who is leading right now
    (4)  2.) Who is winning right now
    (11) 3.) What card was led (is it trump)
    (11) 4.) What card is winning (is it trump)
    (5)  5.) Which team won each trick so far (+1 for "me", 0 for no one (yet), -1 for opponents)
    (12) 6.) Any players confirmed short in any suits
    Total: 47 columns
    """

[1 0 0 0]
[0 0 1 0]
[1 0 0 0 0 0 0 0 0 1 0]
[0 0 0 0 0 1 0 0 0 1 0]
[-1 -1  1 -1  0]
[0 0 0 0 1 0 1 1 1 0 0 0]
Led card: 9S0


In [82]:
# this seems to be working just fine
num_cols = 13
group3_info = dtf.get_group3_info(data.iloc[0], 0, 0)
for i in range(num_cols):
    print(i, ': ', sum(group3_info[i::num_cols]))

print()
print(p3hand)
print()
cards = [c+s for s in list('CDHS') for c in list('9TJQKA')]
for i in range(24):
    print(cards[i], ': ', group3_info[(num_cols*i):(num_cols*(i+1))])
z="""
    #######
    DATA GROUP 3: All card locations (constant order: 9C, 10C, ..., (D), (H), ..., KS, AS)
    #######
    For each card (24):
    (4)  1.) Confirmed in anyone's hand (my hand + ordered up turn card?)
    (4)  2.) Played in a previous trick by someone (maybe later expand this to which prev trick?)
    (3)  3.) Played in CURRENT trick by someone
    (1)  4.) Confirmed buried
    (1)  5.) Is trump
    Total: 312 columns
    """

0 :  0
1 :  0
2 :  1
3 :  5
4 :  0
5 :  0
6 :  0
7 :  0
8 :  0
9 :  0
10 :  0
11 :  0
12 :  7

['QH', '9H', 'AC', '9C', 'TS', 'TH']

9C :  [0 0 0 0 0 0 0 0 0 0 0 0 0]
TC :  [0 0 0 0 0 0 0 0 0 0 0 0 0]
JC :  [0 0 0 1 0 0 0 0 0 0 0 0 0]
QC :  [0 0 0 1 0 0 0 0 0 0 0 0 0]
KC :  [0 0 0 0 0 0 0 0 0 0 0 0 0]
AC :  [0 0 0 0 0 0 0 0 0 0 0 0 0]
9D :  [0 0 0 0 0 0 0 0 0 0 0 0 0]
TD :  [0 0 0 0 0 0 0 0 0 0 0 0 0]
JD :  [0 0 0 1 0 0 0 0 0 0 0 0 1]
QD :  [0 0 0 0 0 0 0 0 0 0 0 0 0]
KD :  [0 0 0 0 0 0 0 0 0 0 0 0 0]
AD :  [0 0 0 0 0 0 0 0 0 0 0 0 0]
9H :  [0 0 0 0 0 0 0 0 0 0 0 0 1]
TH :  [0 0 1 0 0 0 0 0 0 0 0 0 1]
JH :  [0 0 0 1 0 0 0 0 0 0 0 0 1]
QH :  [0 0 0 0 0 0 0 0 0 0 0 0 1]
KH :  [0 0 0 0 0 0 0 0 0 0 0 0 1]
AH :  [0 0 0 0 0 0 0 0 0 0 0 0 1]
9S :  [0 0 0 1 0 0 0 0 0 0 0 0 0]
TS :  [0 0 0 0 0 0 0 0 0 0 0 0 0]
JS :  [0 0 0 0 0 0 0 0 0 0 0 0 0]
QS :  [0 0 0 0 0 0 0 0 0 0 0 0 0]
KS :  [0 0 0 0 0 0 0 0 0 0 0 0 0]
AS :  [0 0 0 0 0 0 0 0 0 0 0 0 0]


In [44]:
# this seems to be working just fine
group4_info = dtf.get_group4_info(data.iloc[0], 0, 0)
print(group4_info, '\n\n', p0hand, '\n\n', (np.argwhere(np.array(group4_info)==1).reshape(-1)+1)%11)

[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0] 

 ['JH', 'JD', 'QC', 'JC', '9S'] 

 [ 4  7  3  9  0  3  8  0  3  7  1 10]


In [221]:
folder = os.path.join(os.getcwd(), 'RLdataset')
first_group = pd.concat([pd.read_csv(os.path.join(folder, '10000_hands_'+str(i).zfill(3)+'.csv'))\
                         for i in notebook.tqdm(range(2))]).reset_index(drop=True)
first_group.head()

  0%|          | 0/2 [00:00<?, ?it/s]

,p0c1,p0c1isD,p0c1isH,p0c1isS,p0c2,p0c2isD,p0c2isH,p0c2isS,p0c3,p0c3isD,p0c3isH,p0c3isS,p0c4,p0c4isD,p0c4isH,p0c4isS,p0c5,p0c5isD,p0c5isH,p0c5isS,p1c1,p1c1isD,p1c1isH,p1c1isS,p1c2,p1c2isD,p1c2isH,p1c2isS,p1c3,p1c3isD,p1c3isH,p1c3isS,p1c4,p1c4isD,p1c4isH,p1c4isS,p1c5,p1c5isD,p1c5isH,p1c5isS,p2c1,p2c1isD,p2c1isH,p2c1isS,p2c2,p2c2isD,p2c2isH,p2c2isS,p2c3,p2c3isD,p2c3isH,p2c3isS,p2c4,p2c4isD,p2c4isH,p2c4isS,p2c5,p2c5isD,p2c5isH,p2c5isS,p3c1,p3c1isD,p3c1isH,p3c1isS,p3c2,p3c2isD,p3c2isH,p3c2isS,p3c3,p3c3isD,p3c3isH,p3c3isS,p3c4,p3c4isD,p3c4isH,p3c4isS,p3c5,p3c5isD,p3c5isH,p3c5isS,TC_power,TC_isD,TC_isH,TC_isS,trump_isD,trump_isH,trump_isS,caller,round,alone,winner1,winner2,winner3,winner4,winner5,points02,points13,played1,played2,played3,played4,played5,played6,played7,played8,played9,played10,played11,played12,played13,played14,played15,played16,played17,played18,played19,played20,result,p0trueid,p1trueid,p2trueid,p3trueid
0,12,1,0,0,4,0,1,0,2,0,1,0,1,0,1,0,1,0,0,1,25,1,0,0,4,0,0,0,3,0,0,0,5,0,1,0,5,0,0,1,10,0,0,0,5,0,0,0,10,0,0,1,3,0,0,1,2,0,0,1,31,0,1,0,15,1,0,0,2,0,0,0,10,0,1,0,4,0,0,1,30,1,0,0,1,0,0,3,1,0,3,3,2,3,3,0,1,QH0,KH1,TS2,AH3,JH3,9D0,KD1,JS2,QS3,9S0,KS1,AS2,AC2,TD3,9H0,JC1,AD3,TH0,QC1,KC2,Single,3,0,1,2
1,35,0,0,1,20,0,0,1,10,0,0,0,4,0,0,0,10,1,0,0,30,0,0,1,3,1,0,0,10,0,1,0,5,0,1,0,3,0,1,0,31,0,0,0,15,0,0,1,4,1,0,0,2,1,0,0,1,0,1,0,25,0,0,1,1,0,0,0,5,1,0,0,1,1,0,0,4,0,1,0,12,0,0,1,0,0,1,0,1,0,2,0,0,3,0,1,0,AC0,AS1,JC2,QH3,QD2,KD3,AD0,JD1,JS0,JH1,TS2,9S3,QC0,KH1,9H2,KS3,9D3,QS0,AH1,TD2,Single,0,1,2,3
2,1,0,0,0,5,1,0,0,4,1,0,0,10,0,1,0,4,0,1,0,25,0,0,1,2,0,0,0,3,1,0,0,1,1,0,0,3,0,1,0,10,0,0,0,5,0,0,0,4,0,0,0,5,0,1,0,1,0,1,0,35,0,0,1,31,0,0,0,15,0,0,1,12,0,0,1,10,1,0,0,30,0,0,1,0,0,1,3,1,1,3,3,3,3,3,0,4,AH0,--1,9H2,9S3,JS3,9C0,--1,QC2,JC3,QD0,--1,KC2,AS3,QH0,--1,KH2,TS3,KD0,--1,AC2,Loner,1,2,3,0
3,2,0,0,1,30,0,0,0,20,0,0,0,15,0,0,0,3,0,1,0,4,0,0,1,12,0,0,0,2,1,0,0,1,1,0,0,1,0,1,0,35,0,0,0,25,0,0,0,4,1,0,0,3,1,0,0,10,0,1,0,5,0,0,1,1,0,0,1,10,1,0,0,5,0,1,0,2,0,1,0,35,0,0,1,0,0,0,0,2,0,2,2,0,0,3,1,0,JH0,9H1,AH2,TH3,JC2,9S3,TC0,9C1,JD2,AD3,QC0,9D1,AC0,TD1,KC2,KH3,TS0,QS1,QD2,KS3,Single,2,3,0,1
4,1,0,0,0,5,1,0,0,4,0,1,0,3,0,1,0,2,0,1,0,31,0,0,0,10,1,0,0,4,1,0,0,2,1,0,0,5,0,1,0,30,0,0,1,15,0,0,1,5,0,0,0,4,0,0,0,10,0,1,0,25,0,0,1,12,0,0,1,10,0,0,0,1,1,0,0,1,0,1,0,35,0,0,1,0,0,1,2,1,0,3,3,3,2,3,0,2,KD0,AD1,TS2,KS3,AC3,9C0,KH1,QC2,JS3,TH0,JC1,AS2,9H3,QH0,TD1,AH2,KC2,9S3,JH0,QD1,EUCHRE,3,0,1,2


In [222]:
first_formatted = dtf.format_data(first_group)

  0%|          | 0/20000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [223]:
os.cpu_count()

12

In [289]:
def get_next_file(datafolder, skipfiles_path):
    with open(skipfiles_path, 'r') as f:
        lines = f.read()
    skipfiles = lines.split('\n')
    for f in os.listdir(datafolder):
        f = os.path.join(datafolder, f)
        if '.csv' not in f:
            continue
        if '_hands_' not in f:
            continue
        if f in skipfiles:
            continue
        return f

def reserve_file(skipfiles_path, file):
    with open(skipfiles_path, 'a') as outfile:
        outfile.write(file+'\n')
def unreserve_file(skipfiles_path, file):
    with open(skipfiles_path, 'r') as f:
        lines = f.read()
    output = '\n'.join([l for l in lines.split('\n') if l!=file])
    with open(skipfiles_path, 'w') as f:
        f.write(output)

In [244]:
def format_onefile(infile):
    df = pd.read_csv(infile)
    data, target = dtf.format_data(df)
    return data, target

In [313]:
from tqdm.auto import tqdm as tqdm

  0%|          | 0/10000000 [00:00<?, ?it/s]

In [262]:
def data_to_tfrecord(data, target, tfrecords_folder, ogfile_name):
    outfile = os.path.join(tfrecords_folder, ogfile_name.split('/')[-1].split('.csv')[0] + '.tfrecord')
    with tf.io.TFRecordWriter(outfile) as tfrec:
        dataset = tf.data.Dataset.from_tensor_slices((np.expand_dims(data, axis=0), np.expand_dims(target, axis=0)))
        for d, t in dataset.take(1):
            example = serialize_example(tf.io.serialize_tensor(d), tf.io.serialize_tensor(t))
        tfrec.write(example)

def serialize_example(data, target):
    feature = {
        'data':bytes_feature(data),
        'target':bytes_feature(target)
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

def bytes_feature(value):
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [301]:
def run_file(datafolder, tfrecords_folder, count=None, dt_stop=None, skipfile_name=None):
    if not os.path.exists(tfrecords_folder): os.mkdir(tfrecords_folder)
    skipfiles_path = os.path.join(tfrecords_folder, 'skipfiles.txt' if skipfile_name is None else skipfile_name)
    with open(skipfiles_path, 'a') as f:
        f.write('')
    
    try:
        if count is not None:
            for i in range(int(count)):
                next_file = get_next_file(datafolder, skipfiles_path)
                print('Starting file %s...' %next_file)
                reserve_file(skipfiles_path, next_file)
                data, target = format_onefile(next_file)
                data_to_tfrecord(data, target, tfrecords_folder, next_file)
        elif dt_stop is not None:
            tstart = time.time()
            while time.time() - tstart < dt_stop:
                next_file = get_next_file(datafolder, skipfiles_path)
                print('Starting file %s...' %next_file)
                reserve_file(skipfiles_path, next_file)
                data, target = format_onefile(next_file)
                data_to_tfrecord(data, target, tfrecords_folder, next_file)
        else:
            # just do one
            next_file = get_next_file(datafolder, skipfiles_path)
            print('Starting file %s...' %next_file)
            reserve_file(skipfiles_path, next_file)
            data, target = format_onefile(next_file)
            data_to_tfrecord(data, target, tfrecords_folder, next_file)
    except KeyboardInterrupt:
        print('Unreserving file %s' %next_file)
        unreserve_file(skipfiles_path, next_file)
        raise KeyboardInterrupt
    except:
        print('Unreserving file %s' %next_file)
        unreserve_file(skipfiles_path, next_file)
        print("Unexpected error:", sys.exc_info()[0])
        raise
    
    print('Done!')

In [300]:
run_file('RLdataset', 'tfrecords')

Starting file RLdataset/10000_hands_365.csv...
Unreserving file RLdataset/10000_hands_365.csv
RLdataset/10000_hands_403.csv


RLdataset/10000_hands_365.csv
Unexpected error: <class 'ModuleNotFoundError'>


ModuleNotFoundError: No module named 'sjdklfjskldf'

In [238]:
z = np.zeros(shape=(20*10000, dtf.COLS_TOTAL), dtype=np.int8)

print(z.shape, z.nbytes)
print(z.nbytes/1024/1024)

(200000, 438) 87600000
83.5418701171875


In [322]:
expected_count_comma = 122
def count_odd_lines(infile):
    with open(infile, 'r') as f:
        lines = f.read()
    
    weird_lines = [[i, len(line.split(','))] for i, line in enumerate(lines.split('\n')) if len(line.split(',')) != expected_count_comma]
    
    return weird_lines

In [317]:
len(count_odd_lines('RLdataset/10000_hands_500.csv'))

64

In [321]:
for i in range(500, 797):
    file = 'RLdataset/10000_hands_' + str(i) + '.csv'
    weird_lines = count_odd_lines(file)
    if len(weird_lines) > 1:
        print(file+':', len(weird_lines), weird_lines[:5])

RLdataset/10000_hands_500.csv: 64 [[1340, 243], [1815, 244], [1870, 244], [1897, 247], [1924, 247]]
RLdataset/10000_hands_611.csv: 5 [[306, 125], [334, 123], [361, 243], [10000, 13], [10001, 1]]
RLdataset/10000_hands_666.csv: 52 [[698, 154], [725, 255], [752, 228], [780, 145], [807, 250]]


In [323]:
for i in range(500, 797):
    file = 'RLdataset/10000_hands_' + str(i) + '.csv'
    weird_lines = count_odd_lines(file)
    if len(weird_lines) > 1:
        print('Re-doing file %s' %file)
        with open(file, 'r') as infile:
            dirty = infile.read()
        
        clean = '\n'.join([line for line in dirty.split('\n') if len(line.split(',')) == expected_count_comma])
        with open(file, 'w') as outfile:
            outfile.write(clean)
        
        weird_lines = count_odd_lines(file)
        if len(weird_lines) > 1:
            print('ERROR IN CLEANING FILE %s' %file)

Re-doing file RLdataset/10000_hands_500.csv
Re-doing file RLdataset/10000_hands_611.csv
Re-doing file RLdataset/10000_hands_666.csv
